In [80]:
import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp
from sklearn.metrics import roc_auc_score, mean_absolute_error

import pandas as pd
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from efficientnet_pytorch import EfficientNet

INPUT_PATH = Path('../input')
TRAIN_PATH = INPUT_PATH / 'idao_dataset' / 'train'
PRIVATE_PATH = INPUT_PATH / 'idao_dataset' / 'private_test'
PUBLIC_PATH = INPUT_PATH / 'idao_dataset' / 'public_test'

RANDOM_SEED = 4444
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(RANDOM_SEED)

## Data prep

In [175]:
mapper1 = {
    1:[1, 0, 0, 0, 0, 0],
    3:[0, 1, 0, 0, 0, 0],
    6:[0, 0, 1, 0, 0, 0],
    10:[0, 0, 0, 1, 0, 0],
    20:[0, 0, 0, 0, 1, 0],
    30:[0, 0, 0, 0, 0, 1],
}

mapper2 = {
    1:0,
    3:1,
    6:2,
    10:3,
    20:4,
    30:5,
}

reverse_mapping = {
    0: 1,
    1: 3,
    2: 6,
    3: 10,
    4: 20,
    5: 30
}

In [2]:
import glob
images = glob.glob(str(TRAIN_PATH / '**/*.png'), recursive=True)

train_images, test_images = train_test_split(images, shuffle = True, random_state = RANDOM_SEED)

In [114]:
def calc_metric(y_binary_true, y_binary_pred, y_reg_true, y_reg_pred):
    '''
    Competition metric
    '''
    
    roc = roc_auc_score(y_binary_true, y_binary_pred)
    mae = mean_absolute_error(y_reg_true, y_reg_pred)
    return 1000 * (roc - mae), roc, mae

In [183]:
class DataGetter(Dataset):
    def __init__(self, image_paths, train=True, transform=None):
 
        self.image_paths = image_paths 
        self.transform=transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image=cv2.imread(self.image_paths[idx])
        if len(self.image_paths[idx].split('_')) == 18:
            particle_class = 0
            particle_energy = mapper2[int(self.image_paths[idx].split('_')[7])]
        else:
            particle_class = 1
            particle_energy = mapper2[int(self.image_paths[idx].split('_')[8])]
        
        sample={
            'image': np.uint8(image), 
            'particle_class': particle_class,
            'particle_energy': particle_energy
            }

        #Applying transformation
        if self.transform:
            sample['image']=self.transform(sample['image'])
            
        return sample

In [184]:
augs = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    ])

In [185]:
transformed_train_data = DataGetter(train_images, train=True, transform=augs)
transformed_test_data = DataGetter(test_images, train=False, transform=augs)

train_dataloader = DataLoader(transformed_train_data, batch_size=32, shuffle=True) #, num_workers=2
test_dataloader = DataLoader(transformed_test_data, batch_size=32, shuffle=True)

## Training classification

In [38]:
import torchvision.models as models

class CNN(nn.Module):
    def __init__(self, pretrained=False):
        super(CNN, self).__init__()
        if pretrained:
            self.model = models.resnet18(pretrained=True)
        else:
            self.model = models.resnet18()
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        
        self.fc0 = nn.Linear(512, 64)
        self.fc1 = nn.Linear(64, 2)  # For classification
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.relu(self.fc0(x))
        particle_class = torch.softmax(self.fc1(x), dim = 1)
        return {'particle_class': particle_class}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_CNN = CNN(False).to(device)

In [22]:
# for item in train_dataloader:
#     img = item['image']
#     break
# model_CNN(img.to(device))

{'particle_class': tensor([[0.4040, 0.5960],
         [0.4580, 0.5420],
         [0.4278, 0.5722],
         [0.4286, 0.5714],
         [0.2901, 0.7099],
         [0.4603, 0.5397],
         [0.4197, 0.5803],
         [0.4545, 0.5455],
         [0.4400, 0.5600],
         [0.4471, 0.5529],
         [0.3924, 0.6076],
         [0.4581, 0.5419],
         [0.4541, 0.5459],
         [0.4577, 0.5423],
         [0.4065, 0.5935],
         [0.4047, 0.5953],
         [0.4482, 0.5518],
         [0.4125, 0.5875],
         [0.4613, 0.5387],
         [0.4008, 0.5992],
         [0.3479, 0.6521],
         [0.4670, 0.5330],
         [0.3692, 0.6308],
         [0.4691, 0.5309],
         [0.4575, 0.5425],
         [0.4541, 0.5459],
         [0.4641, 0.5359],
         [0.4536, 0.5464],
         [0.4570, 0.5430],
         [0.4607, 0.5393],
         [0.4554, 0.5446],
         [0.4503, 0.5497]], device='cuda:0', grad_fn=<SoftmaxBackward>)}

In [39]:
#For binary output:particle_class
criterion_binary= nn.CrossEntropyLoss()
optimizer = optim.Adam(model_CNN.parameters(), lr=1e-3)

In [42]:
def train_model(model, criterion_binary, optimizer, n_epochs=5):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        comp_metric = 0 
        comp_val_metric = 0
        batches = 0
        val_batches = 0
        max_batches = 128
        # train the model #
        model.train()
        for batch_idx, sample_batched in enumerate(train_dataloader):
            if batches > max_batches:
                break
            # importing data and moving to GPU
            image, particle_class = sample_batched['image'].to(device),\
                                             sample_batched['particle_class'].to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            output = model(image)
            label_class = output['particle_class']
    
            
            particle_class = particle_class.squeeze().type(torch.LongTensor).to(device)
            
            
            y_pred_binary = label_class.cpu().detach().numpy()[:, 1]
            y_true_binary = particle_class.cpu().detach().numpy()
            
            roc = roc_auc_score(y_true_binary, y_pred_binary) 

            loss_binary = criterion_binary(label_class, particle_class)
            
            torch.cuda.empty_cache()
            loss = loss_binary
            # back prop
            loss.backward()
            # grad
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            if batch_idx % 50 == 0:
                print('Epoch %d, Batch %d loss: %.6f ROC AUC %.6f' %
                  (epoch, batch_idx + 1, train_loss, roc))
            batches += 1
        # validate the model #
        model.eval()
        
        y_preds = []
        y_trues = []
        for batch_idx, sample_batched in enumerate(test_dataloader):
            if val_batches > max_batches:
                break
            image, particle_class = sample_batched['image'].to(device),\
                                             sample_batched['particle_class'].to(device)
                                              
            
            output = model(image)

            label_class = output['particle_class']

            
            particle_class = particle_class.squeeze().type(torch.LongTensor).to(device)
            
            
            y_pred_binary = label_class.cpu().detach().numpy()[:, 1]
            y_true_binary = particle_class.cpu().detach().numpy()
            
            y_preds.extend(list(y_pred_binary))
            y_trues.extend(list(y_true_binary))
            
            # calculate loss
            loss_binary = criterion_binary(label_class, particle_class)
            
            loss = loss_binary
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            val_batches += 1
        
        roc_val = roc_auc_score(y_trues, y_preds) 
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t ROC AUC {:.6f}'.format(
            epoch, train_loss, valid_loss, roc_val))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model, 'model_classification.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
    # return trained model
    return model

In [43]:
model_conv=train_model(model_CNN, criterion_binary, optimizer)

Epoch 1, Batch 1 loss: 0.545860 ROC AUC 0.855469
Epoch 1, Batch 51 loss: 0.378313 ROC AUC 0.995951
Epoch 1, Batch 101 loss: 0.352131 ROC AUC 1.000000
Epoch: 1 	Training Loss: 0.367238 	Validation Loss: 0.494362 	 ROC AUC 0.800466
Validation loss decreased (inf --> 0.494362).  Saving model ...
Epoch 2, Batch 1 loss: 0.313505 ROC AUC 1.000000
Epoch 2, Batch 51 loss: 0.445201 ROC AUC 1.000000
Epoch 2, Batch 101 loss: 0.390955 ROC AUC 1.000000
Epoch: 2 	Training Loss: 0.378029 	Validation Loss: 0.813779 	 ROC AUC 0.990208
Epoch 3, Batch 1 loss: 0.313365 ROC AUC 1.000000
Epoch 3, Batch 51 loss: 0.318851 ROC AUC 1.000000
Epoch 3, Batch 101 loss: 0.325147 ROC AUC 0.988095
Epoch: 3 	Training Loss: 0.326511 	Validation Loss: 0.315715 	 ROC AUC 0.999895
Validation loss decreased (0.494362 --> 0.315715).  Saving model ...
Epoch 4, Batch 1 loss: 0.343286 ROC AUC 1.000000
Epoch 4, Batch 51 loss: 0.318035 ROC AUC 1.000000
Epoch 4, Batch 101 loss: 0.317860 ROC AUC 1.000000
Epoch: 4 	Training Loss: 0.

## Training regression

In [216]:
import torchvision.models as models

class CNNReg(nn.Module):
    def __init__(self, pretrained=False):
        super(CNNReg, self).__init__()
        if pretrained:
            self.model = models.resnet18(pretrained=True)
        else:
            self.model = models.resnet18()
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        
        self.fc0 = nn.Linear(512, 64)
        self.fc1 = nn.Linear(64, 6)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.relu(self.fc0(x))
        particle_energy = torch.softmax(self.fc1(x), dim = 1)
        return {'particle_energy': particle_energy}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_reg = CNNReg(False).to(device)

In [217]:
criterion_reg= nn.CrossEntropyLoss()
optimizer = optim.Adam(model_reg.parameters(), lr=1e-3)

In [218]:
for item in train_dataloader:
    img = item['image']
    break
model_reg(img.to(device))

{'particle_energy': tensor([[0.1652, 0.1572, 0.1495, 0.2723, 0.1280, 0.1278],
         [0.1819, 0.1663, 0.1501, 0.2096, 0.1449, 0.1472],
         [0.1817, 0.1558, 0.1430, 0.3194, 0.1080, 0.0922],
         [0.1837, 0.1542, 0.1498, 0.2376, 0.1365, 0.1381],
         [0.1796, 0.1576, 0.1513, 0.2174, 0.1470, 0.1470],
         [0.1721, 0.1601, 0.1472, 0.2646, 0.1292, 0.1267],
         [0.1798, 0.1521, 0.1483, 0.2667, 0.1262, 0.1269],
         [0.1868, 0.1590, 0.1481, 0.2203, 0.1441, 0.1418],
         [0.1775, 0.1582, 0.1478, 0.2164, 0.1544, 0.1457],
         [0.1842, 0.1527, 0.1450, 0.2217, 0.1475, 0.1489],
         [0.1639, 0.1497, 0.1394, 0.3509, 0.0969, 0.0992],
         [0.1782, 0.1593, 0.1462, 0.2176, 0.1456, 0.1532],
         [0.1858, 0.1560, 0.1490, 0.2631, 0.1253, 0.1208],
         [0.1706, 0.1598, 0.1436, 0.2291, 0.1471, 0.1497],
         [0.1738, 0.1573, 0.1471, 0.2695, 0.1232, 0.1291],
         [0.1801, 0.1608, 0.1486, 0.2120, 0.1513, 0.1471],
         [0.1892, 0.1580, 0.1494, 0.2

In [219]:
def train_model_reg(model, criterion_reg, optimizer, n_epochs=5):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        comp_metric = 0 
        comp_val_metric = 0
        batches = 0
        val_batches = 0
        max_batches = 128
        # train the model #
        model.train()
        y_preds = []
        y_trues = []
        for batch_idx, sample_batched in enumerate(train_dataloader):
            if batches > max_batches:
                break
            # importing data and moving to GPU
            image, particle_energy = sample_batched['image'].to(device),\
                                             sample_batched['particle_energy'].to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            output = model(image)
            label_energy = output['particle_energy']
    
            
            particle_energy = particle_energy.squeeze().type(torch.LongTensor).to(device)
            
            
            y_pred_reg = label_energy.cpu().detach().numpy()
            y_pred_reg = [reverse_mapping[x] for x in list(np.argmax(y_pred_reg, axis = 1))]
            
            y_true_reg = particle_energy.cpu().detach().numpy()
            y_true_reg = [reverse_mapping[x] for x in list(y_true_reg)]
            
            
            mae = mean_absolute_error(y_true_reg, y_pred_reg) 

            loss_reg = criterion_reg(label_energy, particle_energy)
            
            y_preds.extend(list(y_pred_reg))
            y_trues.extend(list(y_true_reg))
            
            
            torch.cuda.empty_cache()
            loss = loss_reg
            # back prop
            loss.backward()
            # grad
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            if batch_idx % 1 == 0:
                print('Epoch %d, Batch %d loss: %.6f MAE %.6f' %
                  (epoch, batch_idx + 1, train_loss, mae))
            batches += 1
        # validate the model #
        model.eval()
        
        y_preds = []
        y_trues = []
        for batch_idx, sample_batched in enumerate(test_dataloader):
            if val_batches > max_batches:
                break
            # importing data and moving to GPU
            image, particle_energy = sample_batched['image'].to(device),\
                                             sample_batched['particle_energy'].to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            output = model(image)
            label_energy = output['particle_energy']

            
            particle_energy = particle_energy.squeeze().type(torch.LongTensor).to(device)
            
            
            y_pred_reg = label_energy.cpu().detach().numpy()
            y_pred_reg = [reverse_mapping[x] for x in list(np.argmax(y_pred_reg, axis = 1))]
            
            y_true_reg = particle_energy.cpu().detach().numpy()
            y_true_reg = [reverse_mapping[x] for x in list(y_true_reg)]
            
            y_preds.extend(list(y_pred_reg))
            y_trues.extend(list(y_true_reg))
            

            loss_reg = criterion_reg(label_energy, particle_energy)
            
            loss = loss_reg
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            val_batches += 1
        
        mae_val = mean_absolute_error(y_trues, y_preds) 
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t MAE {:.6f}'.format(
            epoch, train_loss, valid_loss, mae_val))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model, 'model_regression.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
    # return trained model
    return model

In [ ]:
model_reg = train_model_reg(model_reg, criterion_reg, optimizer)

Epoch 1, Batch 1 loss: 1.789550 MAE 7.343750
Epoch 1, Batch 2 loss: 1.731750 MAE 3.406250
Epoch 1, Batch 3 loss: 1.669055 MAE 2.093750
Epoch 1, Batch 4 loss: 1.635806 MAE 2.187500
Epoch 1, Batch 5 loss: 1.608848 MAE 2.843750
Epoch 1, Batch 6 loss: 1.602455 MAE 3.125000
Epoch 1, Batch 7 loss: 1.592353 MAE 3.156250
Epoch 1, Batch 8 loss: 1.575077 MAE 2.062500
Epoch 1, Batch 9 loss: 1.564806 MAE 2.593750
Epoch 1, Batch 10 loss: 1.562843 MAE 3.093750
Epoch 1, Batch 11 loss: 1.566602 MAE 3.687500
Epoch 1, Batch 12 loss: 1.562003 MAE 2.812500
Epoch 1, Batch 13 loss: 1.557052 MAE 2.562500
Epoch 1, Batch 14 loss: 1.558689 MAE 3.937500
Epoch 1, Batch 15 loss: 1.555058 MAE 2.875000
Epoch 1, Batch 16 loss: 1.556695 MAE 3.468750
Epoch 1, Batch 17 loss: 1.549032 MAE 2.281250
Epoch 1, Batch 18 loss: 1.544991 MAE 2.843750
Epoch 1, Batch 19 loss: 1.535101 MAE 1.531250
Epoch 1, Batch 20 loss: 1.520038 MAE 0.906250
Epoch 1, Batch 21 loss: 1.515708 MAE 3.500000
Epoch 1, Batch 22 loss: 1.507476 MAE 1.2812

## Sanity checks

In [71]:
model = torch.load('model.pt')
model.eval();

In [72]:
class TestDataGetter(Dataset):
    def __init__(self, image_paths, train=True, transform=None):
 
        self.image_paths = image_paths 
        self.transform=transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image=cv2.imread(self.image_paths[idx])
        
        sample={
            'image': np.uint8(image)
            }

        #Applying transformation
        if self.transform:
            sample['image']=self.transform(sample['image'])
            
        return sample

In [81]:
private_test = glob.glob(str(PRIVATE_PATH / '**/*.png'), recursive=True)
public_test = glob.glob(str(PUBLIC_PATH / '**/*.png'), recursive=True)

In [83]:
private_test_getter = TestDataGetter(private_test, transform=augs)
public_test_getter = TestDataGetter(public_test, transform=augs)

private_test_dataloader = DataLoader(private_test_getter, batch_size=32, shuffle=False) #, num_workers=2
public_test_dataloader = DataLoader(public_test_getter, batch_size=32, shuffle=False)

In [96]:
# public set

public_predictions = []
for batch in public_test_dataloader:
    imgs = batch['image'].to(device)
    preds = model(imgs)
    preds = preds['particle_class'].cpu().detach().numpy()
    preds = np.argmax(preds, axis = 1)
    
    public_predictions.extend(preds)
pd.Series(public_predictions).value_counts()

1    753
0    749
dtype: int64

In [98]:
# private set

private_predictions = []
for batch in private_test_dataloader:
    imgs = batch['image'].to(device)
    preds = model(imgs)
    preds = preds['particle_class'].cpu().detach().numpy()
    preds = np.argmax(preds, axis = 1)
    
    private_predictions.extend(preds)
pd.Series(private_predictions).value_counts()

KeyboardInterrupt: 

## Predict for test